In [1]:
import torch
from torch import nn

不含模型参数的自定义层

In [2]:
class CenteredLayer(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(CenteredLayer, self).__init__(*args, **kwargs)
    
    def forward(self, x):
        return x - x.mean()

可以实例化这个层，然后做前向计算

In [3]:
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

也可以构造复杂的模型

In [5]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

y = net(torch.rand(4, 8))
y.mean().item()

2.7939677238464355e-09

含模型参数的自定义层

In [15]:
class MyListDense(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(MyListDense, self).__init__(*args, **kwargs)
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
    
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)


In [14]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)


MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


这样就可以根据传入的键值来进行不同的前向传播

In [13]:
x = torch.ones(1, 4)
print(net(x, 'linear1'))
print(net(x, 'linear2'))
print(net(x, 'linear3'))

tensor([[ 4.1729,  0.9186, -0.2893, -1.5303]], grad_fn=<MmBackward0>)
tensor([[0.1152]], grad_fn=<MmBackward0>)
tensor([[-1.0906,  1.1879]], grad_fn=<MmBackward0>)


也可以使用自定义层构造模型

In [16]:
net = nn.Sequential(
    MyDictDense(),
    MyListDense(),
)
print(net)
print(net(x))


Sequential(
  (0): MyDictDense(
    (params): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): MyListDense(
    (params): ParameterList(
        (0): Parameter containing: [torch.float32 of size 4x4]
        (1): Parameter containing: [torch.float32 of size 4x4]
        (2): Parameter containing: [torch.float32 of size 4x4]
        (3): Parameter containing: [torch.float32 of size 4x1]
    )
  )
)
tensor([[3.5225]], grad_fn=<MmBackward0>)
